# VAE
我们创建了一个假设的样本 sample_image，并通过编码器模型传递它以获得潜在空间的表示。最后，我们打印出潜在空间的均值、对数方差以及从潜在分布中采样的一个点，
请注意，这个例子没有包含VAE的完整实现，例如解码器、损失函数和训练过程。这里只展示了如何使用编码器来获取一个样本在潜在空间的表示。在实际应用中，你还需要实现解码器并定义一个合适的损失函数来训练整个VAE模型

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses

# 定义编码器
class Sampling(layers.Layer):
    """使用 (z_mean, z_log_var) 采样潜在空间 Z 的层"""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# 编码器模型
latent_dim = 2  # 潜在空间的维度
input_shape = (28, 28)  # 假设输入是28x28的图像

# 建立编码器模型
inputs = tf.keras.Input(shape=input_shape)
x = layers.Flatten()(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(inputs, [z_mean, z_log_var, z], name='encoder')

# 打印模型结构
encoder.summary()

# 假设我们有一个样本
sample_image = tf.random.normal([1] + list(input_shape))

# 使用编码器得到潜在空间的表示
z_mean, z_log_var, z = encoder(sample_image)

# 打印潜在空间的均值、方差和样本表示
print(f"原始样本: {sample_image},{sample_image.shape}")
print(f"Mean of latent space: {z_mean.numpy()}")
print(f"Log variance of latent space: {z_log_var.numpy()}")
print(f"Sample in latent space: {z.numpy()}")


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten_5 (Flatten)            (None, 784)          0           ['input_6[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 64)           50240       ['flatten_5[0][0]']              
                                                                                                  
 dense_11 (Dense)               (None, 64)           4160        ['dense_10[0][0]']               
                                                                                            

# GAN
总结来说，GAN不需要为每个样本计算潜在空间表示的均值和方差，而是依赖于生成器网络直接从潜在空间生成样本

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

# 定义生成器模型
def build_generator(z_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=z_dim))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dense(28*28*1, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# 设置潜在空间的维度
z_dim = 100

# 创建生成器实例
generator = build_generator(z_dim)

# 假设我们有一个潜在空间的随机向量
random_vector = tf.random.normal([1, z_dim])

# 使用生成器生成一个样本
generated_image = generator(random_vector)

# 打印生成的样本
print(generated_image.numpy().shape)


(1, 28, 28, 1)
